First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import torch
import torchvision

CATEGORIES = ['apex','dsad']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, len(CATEGORIES)+1)
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [2]:
model.load_state_dict(torch.load('model_donkey_12_15.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [4]:
torch.save(model_trt.state_dict(), 'donkey_12_15_last_trt.pth')

Load the optimized model by executing the cell below

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('donkey_12_15_last_trt.pth'))

<All keys matched successfully>

Create the racecar class

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [3]:
from jetcam.csi_camera import CSICamera

# liu: 感觉capture_fps = 65 过高，可能导致数据滞留，可将 fps 改为 30 进行调试，09-07号任务
camera = CSICamera(width=224, height=224, capture_fps=30)

In [18]:
camera.realease_camera()

AttributeError: 'CSICamera' object has no attribute 'realease_camera'

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [ ]:
from utils import preprocess
import numpy as np
import time

speed_flag = 0

STEERING_GAIN = 0.75
STEERING_BIAS = 0.00

#car.throttle = 0.22

begin_time = time.time()

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[2])
    # car.throttle = 6.667 *y*y*y - 12.33 *y*y + 7.967 *y - 1.58 # 你车速度0.19
    # car.throttle = -13.33 *y*y*y + 27.33*y*y -18*y + 4.02 # 适配飞哥0.19
    #car.throttle = -33.33 *y*y*y + 65.67 *y*y - 42.23*y + 9.07
    car.throttle = -20 *y*y*y + 40.33*y*y -26.33*y + 5.76
    car.steering = x * STEERING_GAIN + STEERING_BIAS - 0.11
#     end_time = time.time()
#     if end_time - begin_time > 15:
#         speed_flag = 1 - speed_flag
#         begin_time = end_time
#         if speed_flag == 1:
#             car.throttle = 0.18
#         else:
#             car.throttle = 0.24

In [9]:
car.throttle = 0